In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
year=2020 # pulse year
quantiles=[0,0.01,0.05]
years=[2050, 2100, 2200, 2300]

In [ ]:
ssp = xr.open_dataset(
    "/shares/gcp/integration/float32/dscim_input_data/climate/AR6/ar6_fair162_sim_and_medianparams_control_pulse_2030-2040-2050-2060-2070-2080_emis_conc_rf_temp_lambdaeff_emissions-driven_naturalfix_v4.0_Jan212022.nc"
).sel(pulse_year=year, drop=True).pulse_temperature.sel(year=slice(2020,2300))

rff = xr.open_dataset(
    f"/shares/gcp/integration/rff/climate/ar6_rff_iter0-19_fair162_CO2_Fossil_control_pulse_{year}_temp_v5.02_newformat_Jan72022.nc"
).sel(pulse_year=year, drop=True).pulse_temperature.sel(year=slice(2020,2300))

In [ ]:
def ssp_crossing_year(ssp, rff, quantile):

    diff= ssp.quantile(quantile, ['simulation']).min('rcp') - rff.quantile(quantile, ['rff_sp', 'simulation'])
    minyear = diff.where(diff > 0, drop=True).idxmin('year')
    
    return minyear.values

def zero_crossing_year(ssp, rff, quantile):

    rff_val = rff.quantile(quantile, ['rff_sp', 'simulation'])
    minyear = rff_val.where(rff_val >= 0, drop=True).idxmin('year')
    
    return minyear.values

In [ ]:
for q in quantiles:
    print("RFF percentile ", q, "crosses SSP in year :",
          ssp_crossing_year(
              ssp=ssp,
              rff=rff,
              quantile=q
          )
         )

for q in quantiles:
    print("RFF percentile ", q, "crosses 0C in year :",
          zero_crossing_year(
              ssp=ssp,
              rff=rff,
              quantile=q
          )
         )

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))

sns.lineplot(data=ssp.min(['simulation', 'rcp']).to_dataframe().reset_index(),
             x='year',
             y='pulse_temperature',
             ax=ax,
             label='ssp min',
            )

sns.lineplot(data=rff.quantile(quantiles, ['rff_sp', 'simulation']).to_dataframe().reset_index(),
             x='year',
             y='pulse_temperature',
             hue='quantile',
             palette=sns.color_palette('Greys', 3),
             ax=ax
            )

plt.suptitle("GMST over time, RFF quantiles and SSP-RCP min")
ax.set_xlabel("Year")
ax.set_ylabel("GMST")

plt.savefig("/mnt/CIL_integration/rff_diagnostics/v3/rff_ssp_gmst/quantile_timeseries.png", dpi=300, bbox_inches='tight')

In [ ]:
boxplot_rff=rff.sel(year=years).to_dataframe().reset_index()
boxplot_ssp=ssp.sel(year=years).to_dataframe().reset_index()
boxplot_rff['GMST'] = 'rff'
boxplot_ssp['GMST'] = 'ssp'
merged=pd.concat([boxplot_ssp, boxplot_rff])

fig, ax = plt.subplots(figsize=(15,8))

sns.boxplot(data=merged,
            x='pulse_temperature',
            y='year',
            hue='GMST',
            orient='h',
            showfliers=True,
            fliersize=1,
            whis=[5, 95],
            ax=ax
           )

ax.set_ylabel("Year")
ax.set_xlabel("GMST")
fig.suptitle("RFF and SSP-RCP GMST values (IQR, 5/95, outliers)")

plt.savefig("/mnt/CIL_integration/rff_diagnostics/v3/rff_ssp_gmst/boxplot_years.png", dpi=300, bbox_inches='tight')